In [ ]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import datetime
import time as t_util


## Define experiment

In [ ]:
experiment = 'S2'


In [ ]:
dir_data_v8_orig  = '/Trendy/Data/Trendy-v8_' + experiment + '/'
dir_data_v9_orig  = '/Trendy/Data/Trendy-v9_' + experiment + '/'
dir_data_v10_orig = '/Trendy/Data/Trendy-v10_' + experiment + '/'
dir_data_v8_corr  = '/Trendy/Data/Trendy-v8_' + experiment + '_corr/'
dir_data_v9_corr  = '/Trendy/Data/Trendy-v9_' + experiment + '_corr/'
dir_data_v10_corr = '/Trendy/Data/Trendy-v10_' + experiment + '_corr/'
dir_forest        = '/GCB2021_commentary/Data/forest_masks/'
dir_tmp           = '/Trendy/Data/tmp/'
dir_grids         = '/Trendy/Data/grids/'
dir_out_v8        = '/Trendy/Data/SLAND_Trendy-v8_' + experiment + '_LatLon/'
dir_out_v9        = '/Trendy/Data/SLAND_Trendy-v9_' + experiment + '_LatLon/'
dir_out_v10       = '/Trendy/Data/SLAND_Trendy-v10_' + experiment + '_LatLon/'
if not os.path.exists(dir_out_v8):   os.mkdir(dir_out_v8)
if not os.path.exists(dir_out_v9):   os.mkdir(dir_out_v9)
if not os.path.exists(dir_out_v10):  os.mkdir(dir_out_v10)


## Calculate forest fraction and regrid to target file

In [ ]:
models = ['CABLE-POP',                                                                                                                                            # Land cover data from Trendy-v8
          'CLASSIC', 'CLM5.0', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.0', 'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs',  # Land cover data from Trendy-v9
          'CLASSIC-N']                                                                                                                                            # Land cover data from Trendy-v10        


################### FOREST PFTs ###################

PFTs_forest = dict()
#Trendy-v8:
PFTs_forest['CABLE-POP']    = [0, 1, 2, 3]
#Trendy-v9:
PFTs_forest['CLASSIC']      = [1, 2, 3, 4, 5]
PFTs_forest['CLM5.0']       = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['ISAM']         = [1, 2, 3, 4, 5, 14, 15, 16, 17, 18, 20, 24]   # Note that the PFTs 14, 15, 16, 17, 18, 24 for ISAM all have fraction 0 everywhere on the world for S2
PFTs_forest['ISBA-CTRIP']   = []                                            # File contains tree cover fraction
PFTs_forest['JSBACH']       = [3, 4, 5, 6]
PFTs_forest['JULES-ES-1.0'] = [0, 1, 2, 3, 4]
PFTs_forest['LPJ-GUESS']    = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
PFTs_forest['LPJwsl']       = [0, 1, 2, 3, 4, 5, 6, 7]
PFTs_forest['LPX-Bern']     = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['OCN']          = [2, 3, 4, 5, 6, 7, 8, 9]
PFTs_forest['ORCHIDEEv3']   = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['SDGVM']        = [7, 8, 9, 10]
PFTs_forest['VISIT']        = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['YIBs']         = [1, 2, 3]
#Trendy-v10:
PFTs_forest['CLASSIC-N']    = [1, 2, 3, 4, 5]


################### ALL NATURAL PFTs (without crops and pasture) ###################
PFTs_natural = dict()
#Trendy-v8:
PFTs_natural['CABLE-POP']    = [0, 1, 2, 3, 4, 5, 6, 7]
#Trendy-v9:
PFTs_natural['CLASSIC']      = [1, 2, 3, 4, 5, 8, 9]
PFTs_natural['CLM5.0']       = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
PFTs_natural['ISAM']         = [1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 20, 21, 24]  # Note that the PFTs 14, 15, 16, 17, 18, 24 for ISAM all have fraction 0 everywhere on the world for S2
PFTs_natural['ISBA-CTRIP']   = []                                                           # File contains tree cover fraction
PFTs_natural['JSBACH']       = [3, 4, 5, 6, 7, 8, 9, 10]
PFTs_natural['JULES-ES-1.0'] = [0, 1, 2, 3, 4, 5, 8, 11, 12]
PFTs_natural['LPJ-GUESS']    = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
PFTs_natural['LPJwsl']       = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
PFTs_natural['LPX-Bern']     = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
PFTs_natural['OCN']          = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
PFTs_natural['ORCHIDEEv3']   = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14]
PFTs_natural['SDGVM']        = [3, 5, 7, 8, 9, 10]
PFTs_natural['VISIT']        = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
PFTs_natural['YIBs']         = [1, 2, 3, 4, 5, 6]
#Trendy-v10:
PFTs_natural['CLASSIC-N']    = [1, 2, 3, 4, 5, 8, 9]


selection = 'ForestFraction'
selection = 'NaturalLandCoverFraction'

if selection=='ForestFraction':
    PFTs_sel = PFTs_forest
    var_name = 'forest_fraction'
elif selection=='NaturalLandCoverFraction':
    PFTs_sel = PFTs_natural
    var_name = 'natural_land_cover_fraction'


#Define compression level
comp = dict(zlib=True, complevel=2)

#Define resolution
resolution = '360x720-ForestMask'#'360x720'#, '720x1440'

#Loop over models
for model in models:
    
    print(model)
    
    #Define folders
    if model=='CABLE-POP':
        dir_model_orig = dir_data_v8_orig + model + '/'
        dir_model_corr = dir_data_v8_corr + model + '/'
        dir_out = dir_out_v8
    elif model=='CLM5.0' and experiment=='S3':
        dir_model_orig = dir_data_v8_orig + 'CLM5.0_Corrected/'
        dir_model_corr = dir_data_v8_corr + 'CLM5.0_Corrected/'
        dir_out = dir_out_v8
    elif model=='CLASSIC-N':
        dir_model_orig = dir_data_v10_orig + model + '/'
        dir_model_corr = dir_data_v10_corr + model + '/'
        dir_out = dir_out_v10
    else:
        dir_model_orig = dir_data_v9_orig + model + '/'
        dir_model_corr = dir_data_v9_corr + model + '/'
        dir_out = dir_out_v9
        
    #Define variable
    if model=='ISBA-CTRIP':
        variable = 'treeFrac'
    else:
        variable = 'landCoverFrac'
    
    #Get file name
    files_lcf_orig = [dir_model_orig + file for file in os.listdir(dir_model_orig) if '_' + variable in file and '_landCoverFrac_lu' not in file and experiment + '_' in file]
    
    #Check if corrected file exists
    if os.path.exists(dir_model_corr):
        files_lcf_corr = [dir_model_corr + file for file in os.listdir(dir_model_corr) if '_' + variable in file and '_landCoverFrac_lu' not in file and experiment + '_' in file]
        N2 = len(files_lcf_corr)
    else:
        N2 = 0
        
    #Select file name
    if N2>0:  files_lcf = files_lcf_corr
    else:     files_lcf = files_lcf_orig
    
    #Check if file is unique
    if len(files_lcf)!=1:  sys.exit('Filename not unique')
    else:                  file_lcf = files_lcf[0]
    
    print(file_lcf)
    
    #Read data
    data_lcf = xr.open_dataset(file_lcf, use_cftime=True)

    #Define variable name for time
    if model in ['ISBA-CTRIP', 'ORCHIDEEv3']:
        data_lcf = data_lcf.rename({'time_counter': 'time'})
    
    #Rename pfts, such that all models have dimension name 'PFT'
    if 'pfts' in data_lcf.dims:       data_lcf = data_lcf.rename({'pfts': 'PFT'})
    elif 'pft' in data_lcf.dims:      data_lcf = data_lcf.rename({'pft': 'PFT'})
    elif 'veget' in data_lcf.dims:    data_lcf = data_lcf.rename({'veget': 'PFT'})
    elif 'vegtype' in data_lcf.dims:  data_lcf = data_lcf.rename({'vegtype': 'PFT'})
    elif 'ntiles' in data_lcf.dims:   data_lcf = data_lcf.rename({'ntiles': 'PFT'})

    #Define output file name
    fname_out = dir_out + model + '_' + experiment + '_' + selection + '.nc'
    if os.path.exists(fname_out):  os.remove(fname_out)
    
    #Calculate forset fraction
    if model=='ISBA-CTRIP':
        
        #Rename tree fraction to forest fraction (or natural land cover fraction)
        data_ForFrac = data_lcf.rename({'treeFrac': var_name})

    else:

        #Select forest (or natural land cover) PFTs for each model and sum over them, and rename variable
        data_ForFrac = data_lcf.sel(PFT=PFTs_sel[model]).sum('PFT')
        data_ForFrac = data_ForFrac.rename({'landCoverFrac': var_name})
    
    #Replace NaNs by 0
    data_ForFrac = data_ForFrac.fillna(0)
    
    #Drop variable 'time_bnds'
    if 'time_bnds' in data_ForFrac.data_vars:
        data_ForFrac = data_ForFrac.drop('time_bnds')
    
    #Convert values [0, 100] -> [0, 1]
    if data_ForFrac[var_name].max()>50:
        data_ForFrac = data_ForFrac / 100

    #Check if land cover data has time dimension
    del_tmp = False
    if 'time' in data_ForFrac.coords:
        
        #Calculate yearly average of monthly data
        if len(data_ForFrac.time)==1:
            data_ForFrac = data_ForFrac.isel(time=0)
        
        elif np.mean(np.diff(data_ForFrac['time']))<datetime.timedelta(days=180):
            
            #Save data in temporary file
            file_lcf_tmp = dir_tmp + model + '_landCoverFrac_tmp.nc'
            data_ForFrac.to_netcdf(file_lcf_tmp)
            
            #Calculate yearly average and save in file
            os.system("cdo -yearmean " + file_lcf_tmp + " " + fname_out)      
            del_tmp = True    
    
    #Rename units for lat and lon for LPJwsl
    if model=='LPJwsl':
        data_ForFrac['lon'].attrs['units'] = 'degrees east'
        data_ForFrac['lat'].attrs['units'] = 'degrees north'
    
    #Save in file
    if del_tmp==False:
        encoding = {var: comp for var in data_ForFrac.data_vars}
        data_ForFrac.to_netcdf(fname_out, encoding=encoding)
    
    #Grid file name 
    file_grid = dir_grids + 'grid_xy_' + resolution

    #Remap forest or natural land cover) fraction data
    fname_regrid = fname_out[0:-3] + '_regrid' + resolution + '.nc'
    if os.path.exists(fname_regrid): os.remove(fname_regrid)
    os.system('cdo -z zip_2 remapcon,' + file_grid + ' ' + fname_out + ' ' + fname_regrid)

    #Remove temporary files
    if del_tmp==True:  os.remove(file_lcf_tmp)


## Get PFT information

In [ ]:

#################### TRENDY-v8 #########################

print('CABLE-POP')
fname = '/Trendy-v8_S2/CABLE-POP/CABLE-POP_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).landCoverFrac.attrs['pft_lut'])
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')


#################### TRENDY-v9 #########################


print('CLASSIC')
fname = '/Trendy-v9_S2/CLASSIC/CLASSIC_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).PFT.attrs['long_name'])
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('CLM5.0')
fname = '/Trendy-v9_S2_corr/CLM5.0/CLM5.0_S2_landCoverFrac.nc'
print('See Table 2.1 in https://www.cesm.ucar.edu/models/cesm2/land/CLM50_Tech_Note.pdf')
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('ISAM')
fname = '/Trendy-v9_S2_corr/ISAM/ISAM_S2_landCoverFrac.nc'
print('See Appendix B in https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi5t_m7j7D5AhWYYPEDHQzjDTMQFnoECAIQAQ&url=https%3A%2F%2Fwww.atmos.illinois.edu%2F~sshu3%2Fmodel%2FISAM_description.docx&usg=AOvVaw0k--gnF922PU9--WL4OqOE')
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('ISBA-CTRIP')
fname = '/Trendy-v9_S2_corr/ISBA-CTRIP/ISBA-CTRIP_S2_treeFrac.nc'
print('This file contains tree fraction ==> I assume it to be forest fraction')
print('')

print('JSBACH')
fname = '/Trendy-v9_S2/JSBACH/JSBACH_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).attrs['comment'])
print(xr.open_dataset(fname, decode_times=False).ntiles.values)
print('')

print('JULES-ES-1.0')
fname = '/Trendy-v9_S2/JULES-ES-1.0/JULES-ES-1.0_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).attrs['vegtype'])
print(xr.open_dataset(fname, decode_times=False).vegtype.values)
print('')

print('LPJ-GUESS')
fname = '/Trendy-v9_S2/LPJ-GUESS/LPJ-GUESS_S2_landCoverFrac.read_the_readme.nc'
print(xr.open_dataset(fname, decode_times=False).attrs)
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('LPJwsl')
fname = '/Trendy-v9_S2/LPJwsl/LPJwsl_S2_landCoverFrac.nc'
print('See Supplementary Table S1 of https://gmd.copernicus.org/articles/14/2575/2021/gmd-14-2575-2021-discussion.html')
print(xr.open_dataset(fname, decode_times=False).pft.values)
print('')

print('LPX-Bern')
fname = '/Trendy-v9_S2/LPX-Bern/LPX-Bern_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).PFT.attrs['long_name'])
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('OCN')
fname = '/Trendy-v9_S2/OCN/OCN_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).landCoverFrac.attrs)
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('ORCHIDEEv3')
fname = '/Trendy-v9_S2/ORCHIDEEv3/ORCHIDEEv3_S2_landCoverFrac.nc'
print('Check: https://orchidas.lsce.ipsl.fr/dev/lccci/orc_15pft.php')
print(xr.open_dataset(fname, decode_times=False).veget.values)
print('')

print('SDGVM')
fname = '/Trendy-v9_S2/SDGVM/SDGVM_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).attrs['PFTs'])
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')

print('VISIT')
fname = '/Trendy-v9_S2/VISIT/VISIT_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).attrs['vegtype'])
print(xr.open_dataset(fname, decode_times=False).vegtype.values)
print('')

print('YIBs')
fname = '/Trendy-v9_S2/YIBs/YIBs_S2_Annual_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).attrs)
print(xr.open_dataset(fname, decode_times=False).pfts.values)
print('')


#################### TRENDY-v10 #########################


print('CLASSIC-N')

fname = '/Trendy-v10_S2/CLASSIC-N/CLASSIC-N_S2_landCoverFrac.nc'
print(xr.open_dataset(fname, decode_times=False).PFT.attrs['long_name'])
print(xr.open_dataset(fname, decode_times=False).PFT.values)
print('')




################### FOREST PFTs ###################

PFTs_forest = dict()
#Trendy-v8:
PFTs_forest['CABLE-POP']    = [0, 1, 2, 3]
#Trendy-v9:
PFTs_forest['CLASSIC']      = [1, 2, 3, 4, 5]
PFTs_forest['CLM5.0']       = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['ISAM']         = [1, 2, 3, 4, 5, 14, 15, 16, 17, 18, 20, 24]   # Note that the PFTs 14, 15, 16, 17, 18, 24 for ISAM all have fraction 0 everywhere on the world for S2
PFTs_forest['ISBA-CTRIP']   = []                                            # File contains tree cover fraction
PFTs_forest['JSBACH']       = [3, 4, 5, 6]
PFTs_forest['JULES-ES-1.0'] = [0, 1, 2, 3, 4]
PFTs_forest['LPJ-GUESS']    = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
PFTs_forest['LPJwsl']       = [0, 1, 2, 3, 4, 5, 6, 7]
PFTs_forest['LPX-Bern']     = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['OCN']          = [2, 3, 4, 5, 6, 7, 8, 9]
PFTs_forest['ORCHIDEEv3']   = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['SDGVM']        = [7, 8, 9, 10]
PFTs_forest['VISIT']        = [1, 2, 3, 4, 5, 6, 7, 8]
PFTs_forest['YIBs']         = [1, 2, 3]
#Trendy-v10:
PFTs_forest['CLASSIC-N']    = [1, 2, 3, 4, 5]


################### ALL NATURAL PFTs (without crops and pasture) ###################
PFTs_natural = dict()
#Trendy-v8:
PFTs_natural['CABLE-POP']    = [0, 1, 2, 3, 4, 5, 6, 7]
#Trendy-v9:
PFTs_natural['CLASSIC']      = [1, 2, 3, 4, 5, 8, 9]
PFTs_natural['CLM5.0']       = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
PFTs_natural['ISAM']         = [1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 20, 21, 24]  # Note that the PFTs 14, 15, 16, 17, 18, 24 for ISAM all have fraction 0 everywhere on the world for S2
PFTs_natural['ISBA-CTRIP']   = []                                                           # File contains tree cover fraction
PFTs_natural['JSBACH']       = [3, 4, 5, 6, 7, 8, 9, 10]
PFTs_natural['JULES-ES-1.0'] = [0, 1, 2, 3, 4, 5, 8, 11, 12]
PFTs_natural['LPJ-GUESS']    = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
PFTs_natural['LPJwsl']       = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
PFTs_natural['LPX-Bern']     = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
PFTs_natural['OCN']          = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
PFTs_natural['ORCHIDEEv3']   = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14]
PFTs_natural['SDGVM']        = [3, 5, 7, 8, 9, 10]
PFTs_natural['VISIT']        = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
PFTs_natural['YIBs']         = [1, 2, 3, 4, 5, 6]
#Trendy-v10:
PFTs_natural['CLASSIC-N']    = [1, 2, 3, 4, 5, 8, 9]

#Models without land cover fraction:
# 'DLEM'
# 'IBIS'
# 'ORCHIDEE' (not important because not used in GCB2021)
